<img src="images/cads-logo.png" style="height: 100px;padding-top:5px" align=left> <img src="images/apache_spark.png" style="height: 20%;width:20%; padding-top:0px" align=right>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
cd "/content/drive/MyDrive/UM Lecture/CADS/13 BDA with Apache Spark 2day"

/content/drive/MyDrive/UM Lecture/CADS/13 BDA with Apache Spark 2day


# Apache Spark Dataframe Exercise

In this exercise, we are going to get some insights on stock market data. We use `walmart_stock.csv` file as our dataset to analyse the data.

### 1- Create an Apache Spark Session

In [2]:
!pip install pyspark


     |████████████████████████████████| 204.2MB 59kB/s 
     |████████████████████████████████| 204kB 39.3MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612242 sha256=98ea17810d9b6e016c7365dfe324819b572d96e8ec0dd9257b3c20a765f8c808
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
import os
import pandas as pd
import numpy as np

In [6]:
spark

### 2- Load the `walmart_stock.csv` file into a dataframe and infer the data schema

In [9]:
pd.read_csv("walmart_stock.csv")

,Date,Open,High,Low,Close,Volume,Adj Close
0,2012-01-03,59.970001,61.060001,59.869999,60.330002,12668800,52.619235
1,2012-01-04,60.209999,60.349998,59.470001,59.709999,9593300,52.078475
2,2012-01-05,59.349998,59.619999,58.369999,59.419998,12768200,51.825539
3,2012-01-06,59.419998,59.450001,58.869999,59.000000,8069400,51.459220
4,2012-01-09,59.029999,59.549999,58.919998,59.180000,6679300,51.616215
...,...,...,...,...,...,...,...
1253,2016-12-23,69.430000,69.750000,69.360001,69.540001,4803900,69.032411
1254,2016-12-27,69.300003,69.820000,69.250000,69.699997,4435700,69.191240
1255,2016-12-28,69.940002,70.000000,69.260002,69.309998,4875700,68.804087
1256,2016-12-29,69.209999,69.519997,69.120003,69.260002,4298400,68.754456


In [82]:
df1 = spark.read.format('csv').option('header','true').option('inferSchema','true').load("walmart_stock.csv")


In [ ]:
## MC
# df = spark.read.csv("walmart_stock.csv",header=True,inferSchema=True)

### 3- Display the column names and print the dataframe schema

In [83]:
df1.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



### 4- Print out the first five rows of the data

In [84]:
df1.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



### 5- Use `describe()` method to get statistical information on the data 

In [85]:
df1.describe().show()

+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|      Date|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+----------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|      1258|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean|      null| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|      null|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|2012-01-03|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|2016-12-30|         90.800003|        90.970001|            89.25|        90.4700

In [113]:
## MC
des_df = df1.describe()

### 6- Use `format_number` function to format the numbers for just showing up to two decimal places. 
[format_number() documentation](http://spark.apache.org/docs/latest/api/python/pyspark.sql.html?highlight=format_number#pyspark.sql.functions.format_number)

In [ ]:
from pyspark.sql import functions

df1.select([functions.format_number(c,3).alias(c) for c in df1]).show


In [114]:
## MC ###
des_df.printSchema()

root
 |-- summary: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Adj Close: string (nullable = true)
 |-- HV Ratio: string (nullable = true)



In [120]:
from pyspark.sql.functions import format_number

des_df.select(des_df['summary'],des_df['Date'],
              format_number(des_df['Open'].cast('float'),2).alias('Open'),
              format_number(des_df['High'].cast('float'),2).alias('High'),
              format_number(des_df['Low'].cast('float'),2).alias('Low'),
              format_number(des_df['Close'].cast('float'),2).alias('Close'),
              format_number(des_df['Volume'].cast('float'),2).alias('Volume'),
              format_number(des_df['Adj Close'].cast('float'),2).alias('Adj Close'),
              ).show()



+-------+----------+--------+--------+--------+--------+-------------+---------+
|summary|      Date|    Open|    High|     Low|   Close|       Volume|Adj Close|
+-------+----------+--------+--------+--------+--------+-------------+---------+
|  count|      1258|1,258.00|1,258.00|1,258.00|1,258.00|     1,258.00| 1,258.00|
|   mean|      null|   72.36|   72.84|   71.92|   72.39| 8,222,093.50|    67.24|
| stddev|      null|    6.77|    6.77|    6.74|    6.76| 4,519,781.00|     6.72|
|    min|2012-01-03|   56.39|   57.06|   56.30|   56.42| 2,094,900.00|    50.36|
|    max|2016-12-30|   90.80|   90.97|   89.25|   90.47|80,898,096.00|    84.91|
+-------+----------+--------+--------+--------+--------+-------------+---------+



### 7- Create a new column called HV Ratio on a new dataframe that returns the ratio of the High Price versus volume of stock traded for a day.

In [87]:
from pyspark.sql.functions import lit
from pyspark.sql.types import StringType


In [88]:
df1 = df1.withColumn('HV Ratio',df1["High"]/df1["Volume"])

In [89]:
df1.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+--------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|            HV Ratio|
+----------+------------------+---------+---------+------------------+--------+------------------+--------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|7.367338463826307E-6|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|8.915604778943901E-6|
+----------+------------------+---------

### 8- What day had the Peak High in Price?

In [90]:
df1.describe("High").show()

+-------+-----------------+
|summary|             High|
+-------+-----------------+
|  count|             1258|
|   mean|72.83938807631165|
| stddev|6.768186808159218|
|    min|        57.060001|
|    max|        90.970001|
+-------+-----------------+



In [91]:
df1.agg({'High':'max'}).show()

+---------+
|max(High)|
+---------+
|90.970001|
+---------+



In [92]:
df1.filter(df1["High"] == 90.970001).show()

+----------+---------+---------+-----+---------+-------+---------+--------------------+
|      Date|     Open|     High|  Low|    Close| Volume|Adj Close|            HV Ratio|
+----------+---------+---------+-----+---------+-------+---------+--------------------+
|2015-01-13|90.800003|90.970001|88.93|89.309998|8215400|83.825448|1.107310672639189...|
+----------+---------+---------+-----+---------+-------+---------+--------------------+



In [127]:
### MC
df1.orderBy(df1.High.desc()).head()[0]

'2015-01-13'

In [128]:
# try chg bit
df1.orderBy("High").head()[0]


'2015-11-13'

### 9-What is the mean of the Close column?

In [93]:
df1.describe("Close").show()

+-------+-----------------+
|summary|            Close|
+-------+-----------------+
|  count|             1258|
|   mean|72.38844998012726|
| stddev|6.756859163732991|
|    min|        56.419998|
|    max|        90.470001|
+-------+-----------------+



In [94]:
df1.agg({'Close':'mean'}).show()

+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



### 10- How many days was the Close lower than 70 USD?

In [95]:
df1.filter(df1["Close"] < 70).count()

397

In [126]:
# MC
df1.filter("Close < 70").count()

397

### 11-What percentage of the time was the High greater than 80 USD ?
#### In other words, (Number of High Days>80)/(Total Days in the dataframe)

In [96]:
df1.filter(df1["High"] > 80).count()/df1.count()

0.09141494435612083

In [129]:
# MC - other ways
df1.filter(df1.High > 80).count()/df1.count()*100

9.141494435612083

### 12-What is the correlation between High and Volume?

In [97]:
df1.stat.corr('High','Volume')

-0.3384326061737161

In [131]:
## MC
from pyspark.sql.functions import corr
df1.select(corr("High","Volume")).show()

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



### 13- What is the max High per year (use GroupBy)?

In [157]:
df2 = df1.withColumn('Year',functions.year('Date'))
df2.show(4)

+----------+------------------+---------+---------+------------------+--------+------------------+--------------------+----+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|            HV Ratio|Year|
+----------+------------------+---------+---------+------------------+--------+------------------+--------------------+----+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|2012|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|2012|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|2012|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|7.367338463826307E-6|2012|
+----------+------------------+---------+---------+------------------+--------+------------------+--------------------+----+


In [135]:
## MC ### 
from pyspark.sql.functions import max,year

year_df = df1.withColumn('Year',year(df1['Date']))

In [137]:
year_df.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+--------------------+----+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|            HV Ratio|Year|
+----------+------------------+---------+---------+------------------+--------+------------------+--------------------+----+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|4.819714653321546E-6|2012|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|6.290848613094555E-6|2012|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|4.669412994783916E-6|2012|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|7.367338463826307E-6|2012|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|8.915604778943901E-6|2012|


In [139]:
max_df=year_df.groupBy('Year').max()
max_df.show()

+----+-----------------+---------+---------+----------+-----------+-----------------+--------------------+---------+
|Year|        max(Open)|max(High)| max(Low)|max(Close)|max(Volume)|   max(Adj Close)|       max(HV Ratio)|max(Year)|
+----+-----------------+---------+---------+----------+-----------+-----------------+--------------------+---------+
|2015|        90.800003|90.970001|    89.25| 90.470001|   80898100|84.91421600000001|2.460931126147897...|     2015|
|2013|        81.209999|81.370003|    80.82| 81.209999|   25683700|        73.929868|3.734784381116044E-5|     2013|
|2014|87.08000200000001|88.089996|86.480003| 87.540001|   22812400|81.70768000000001|3.494261176659442...|     2014|
|2012|        77.599998|77.599998|76.690002| 77.150002|   38007300|        68.568371|2.368837716882401...|     2012|
|2016|             74.5|75.190002|73.629997| 74.300003|   35076700|        73.233524|1.712227308529861...|     2016|
+----+-----------------+---------+---------+----------+---------

In [143]:
max_df.select('Year','max(High)').show()

+----+---------+
|Year|max(High)|
+----+---------+
|2015|90.970001|
|2013|81.370003|
|2014|88.089996|
|2012|77.599998|
|2016|75.190002|
+----+---------+



### 14- What is the average Close for each Calendar Month (close price for Jan,Feb, Mar, etc)?


In [141]:
from pyspark.sql.functions import month,mean

In [142]:
month_df = df1.withColumn('Month',month(df1['Date']))


In [145]:
mean_df=month_df.groupBy('Month').mean("Close")
mean_df.show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|   12|72.84792478301885|
|    1|71.44801958415842|
|    6| 72.4953774245283|
|    3|71.77794377570092|
|    5|72.30971688679247|
|    9|72.18411785294116|
|    4|72.97361900952382|
|    8|73.02981855454546|
|    7|74.43971943925233|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|    2|  71.306804443299|
+-----+-----------------+



In [147]:
## MC 
from pyspark.sql.functions import month,mean
month_df = df1.withColumn('Month',month('Date'))
month_avg = month_df.select('Month','Close').groupBy('Month').mean()
month_avg.show(5)

+-----+----------+-----------------+
|Month|avg(Month)|       avg(Close)|
+-----+----------+-----------------+
|   12|      12.0|72.84792478301885|
|    1|       1.0|71.44801958415842|
|    6|       6.0| 72.4953774245283|
|    3|       3.0|71.77794377570092|
|    5|       5.0|72.30971688679247|
+-----+----------+-----------------+
only showing top 5 rows



In [152]:
month_avg.select("Month",'avg(Close)').orderBy("Month",ascending = True).show()

+-----+-----------------+
|Month|       avg(Close)|
+-----+-----------------+
|    1|71.44801958415842|
|    2|  71.306804443299|
|    3|71.77794377570092|
|    4|72.97361900952382|
|    5|72.30971688679247|
|    6| 72.4953774245283|
|    7|74.43971943925233|
|    8|73.02981855454546|
|    9|72.18411785294116|
|   10|71.57854545454543|
|   11| 72.1110893069307|
|   12|72.84792478301885|
+-----+-----------------+



#### Well Done!